In [1]:
import pandas as pd
from nltk.corpus import stopwords
import string
import numpy as np
from nltk.stem.porter import PorterStemmer

In [2]:
df1=pd.read_csv(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\100.csv')
df2=pd.read_csv(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\101.csv')

In [3]:
df1.head()

,commands
0,open door
1,open the door
2,please open door
3,please open the door
4,Help me open door


In [4]:
df2.head()

,target
0,1
1,1
2,1
3,1
4,1


In [5]:
df=df1.join(df2)
df.head()

,commands,target
0,open door,1
1,open the door,1
2,please open door,1
3,please open the door,1
4,Help me open door,1


In [6]:
def clean_text(text):
    "Pre-process the document. Pass document as a string argument"
    # split into tokens by white space
    tokens = text.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    #print(tokens)
    commands=" ".join(tokens)
    return commands

In [7]:
df.head()

,commands,target
0,open door,1
1,open the door,1
2,please open door,1
3,please open the door,1
4,Help me open door,1


In [8]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [9]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the texts
vec_text = df['commands']
tokenizer.fit_on_texts(vec_text)

In [10]:
# encode training data set
X= tokenizer.texts_to_matrix(vec_text, mode='binary')
print(X.shape)
print(X[:5])

(5600, 13)
[[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]]


In [11]:
Y=np.array(df['target'])
Y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y ,test_size=0.25, random_state=42)

In [13]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, Y_train)

# Predicting the Test set results
y_pred = naive_bayes_classifier.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[376,   0,   0,   0],
       [  0, 358,   0,   0],
       [  0,   0, 325,   0],
       [  0,   0,   0, 341]], dtype=int64)

In [14]:
print('Test Accuracy: %.2f' % (np.sum(cm.diagonal())/np.sum(cm)))
cm_train = confusion_matrix(Y_train, naive_bayes_classifier.predict(X_train))
print('Train Accuracy: %.2f' % (np.sum(cm.diagonal())/np.sum(cm)))

Test Accuracy: 1.00
Train Accuracy: 1.00


## Prediction

In [15]:
import re #Regular expressionas
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import speech_recognition as sr
from gtts import gTTS #Google text to speech
from pygame import mixer
mixer.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
do="Door opened"
dc="Door closed"
mo="Maps opened"
mc="Maps closed"
tts = gTTS(text=str(do), lang='en')
tts.save(r"C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\do.mp3")
tts = gTTS(text=str(dc), lang='en')
tts.save(r"C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\dc.mp3")
tts = gTTS(text=str(mo), lang='en')
tts.save(r"C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\mo.mp3")
tts = gTTS(text=str(mc), lang='en')
tts.save(r"C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\mc.mp3")

In [24]:
def stotex():
    r = sr.Recognizer() #Recognizer instance
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=2) #handle noise
        print("Your command please!")
        audio = r.listen(source) #listen to audio via microphone
 

    try:
        response = r.recognize_google(audio) #recognizing the audio in english (default language)
        print("I think you said '" + response + "'")
        tts = gTTS(text=str(response)+ '  ' +'    done', lang='en')
        tts.save(r"C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\response1.mp3")
 
    except sr.UnknownValueError:
        print("Sorry could not understand audio")
    return response

In [25]:
def predict_sentiment(review):
    # clean
    tokens = clean_text(review)
    encoded = tokenizer.texts_to_matrix([tokens], mode='binary')
    print(encoded)
    # prediction
    prediction = naive_bayes_classifier.predict(encoded)
    return prediction

In [27]:
# Turning on the Voice assistance
r = sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source)
    print(".....")
    audio = r.listen(source)
try:
    response = r.recognize_google(audio)
    print(response)
    
    if(response=='hi'): #Entering speech assistance
        print("Welcome to automated voice assistance!")
        mixer.music.load(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\welcome.mp3')
        mixer.music.play()
        user_response= stotex() #input
        user_response=clean_text(user_response) #Preprocessing
        user_response=predict_sentiment(user_response) #Predection
        
        if(user_response==1):
            mixer.music.load(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\do.mp3')
            mixer.music.play()
            print(do)
        if(user_response==2):
            mixer.music.load(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\dc.mp3')
            mixer.music.play()
            print(dc)
        if(user_response==3):
            mixer.music.load(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\mo.mp3')
            mixer.music.play()
            print(mo)
        if(user_response==4):
            mixer.music.load(r'C:\Users\rakesh.amman\Desktop\Projects\Voice Assistance_Commands\mc.mp3')
            mixer.music.play()
            print(mc)
            
        
except sr.UnknownValueError:
        print()
        





.....
hi
Welcome to automated voice assistance!
Your command please!
I think you said 'please close door and open maps'
[[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Door closed
